In [36]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, sum

spark = SparkSession.builder \
.appName("read")  \
.getOrCreate()

## Описание структуры файла:

- **ip** - IP-адрес клиента
- **timestamp** - Временная метка запроса
- **method** - HTTP-метод (GET, POST, etc.)
- **url** - URL запроса
- **response_code** - Код ответа (200, 404, etc.)
- **response_size** - Размер ответа в байтах

## Задание. Провести анализ информации:

1. Сгруппировать данные по IP и посчитать количество запросов для каждого IP,
   вывести 10 самых активных IP.
2. Сгруппировать данные по HTTP-методу и посчитать количество запросов для каждого метода.
3. Профильтровать и посчитать количество запросов с кодом ответа 404.
4. Сгруппировать данные по дате и просуммировать размер ответов, сортировать по дате.

In [37]:
# Прочитам CSV файл
df = spark.read.csv("/content/web_server_logs.csv", header=True, inferSchema=True)
df = df.withColumn("date", df["timestamp"].cast('date'))
df.printSchema()
df.show(5)

root
 |-- ip: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- method: string (nullable = true)
 |-- url: string (nullable = true)
 |-- response_code: integer (nullable = true)
 |-- response_size: integer (nullable = true)
 |-- date: date (nullable = true)

+--------------+-------------------+------+--------------------+-------------+-------------+----------+
|            ip|          timestamp|method|                 url|response_code|response_size|      date|
+--------------+-------------------+------+--------------------+-------------+-------------+----------+
|118.23.149.206|2024-06-01 19:04:26|DELETE|                main|          200|         2544|2024-06-01|
| 106.31.106.21|2024-05-27 21:17:47|DELETE|                 app|          404|         5333|2024-05-27|
| 120.194.10.75|2024-05-17 06:26:38|DELETE|categories/explor...|          404|         5383|2024-05-17|
| 92.162.183.20|2024-02-02 06:55:10|   GET| wp-content/tag/main|          500|         1469|2

### 1) Сгруппируем данные по IP и посчитайтаем количество запросов для каждого IP, выведем 10 самых активных IP:

In [38]:
df_ip_count = (df.groupBy("ip").agg(count("ip")
             .alias("request_count")).sort(count("ip").desc()))
print("Top 10 active IP addresses:")
df_ip_count.show(10)

Top 10 active IP addresses:
+--------------+-------------+
|            ip|request_count|
+--------------+-------------+
|25.157.200.204|            2|
|144.251.96.229|            1|
|  4.117.73.242|            1|
|169.214.84.160|            1|
| 10.151.10.205|            1|
|  1.66.211.250|            1|
| 47.23.112.152|            1|
| 4.194.191.132|            1|
| 45.151.23.246|            1|
|65.186.114.185|            1|
+--------------+-------------+
only showing top 10 rows



### 2) Сгруппируем данные по HTTP-методу и посчитаем количество запросов для каждого метода.

In [39]:
df_method_count = (df.groupBy("method").agg(count("method")
                  .alias("method_count")).sort(count("ip").desc()))
print("Request count by HTTP method:")
df_method_count.show()

Request count by HTTP method:
+------+------------+
|method|method_count|
+------+------------+
|   PUT|       25283|
|  POST|       25072|
|DELETE|       24859|
|   GET|       24786|
+------+------------+



### 3) Профильтруем и посчитаем количество запросов с кодом ответа 404.

In [41]:
count_response_code_404 = (df.where(df.response_code == 404).count())
print(f"Number of 404 response codes: {count_response_code_404}")

Number of 404 response codes: 24880


### 4) Сгруппируем данные по дате и просуммируем размер ответов, сортируем по дате.

In [40]:
df_date_count = (df.groupBy("date").agg(sum("response_size"))
                .alias("total_response_size").sort(df.date.asc()))
print("Total response_size by day:")
df_date_count.show()

Total response_size by day:
+----------+------------------+
|      date|sum(response_size)|
+----------+------------------+
|2024-01-01|           1498673|
|2024-01-02|           1364779|
|2024-01-03|           1467030|
|2024-01-04|           1660047|
|2024-01-05|           1471441|
|2024-01-06|           1481402|
|2024-01-07|           1597959|
|2024-01-08|           1459232|
|2024-01-09|           1519743|
|2024-01-10|           1392837|
|2024-01-11|           1541038|
|2024-01-12|           1525736|
|2024-01-13|           1591904|
|2024-01-14|           1495506|
|2024-01-15|           1740182|
|2024-01-16|           1446578|
|2024-01-17|           1471862|
|2024-01-18|           1464171|
|2024-01-19|           1296133|
|2024-01-20|           1496959|
+----------+------------------+
only showing top 20 rows

